In [1662]:
import numpy
from csv import reader
from collections import defaultdict
from itertools import chain, combinations

### Konstruksi FP-Tree

In [1663]:
class Node:
    def __init__(self, itemName, frequency, parentNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentNode
        self.children = {}
        self.next = None

    def increment(self, frequency):
        self.count += frequency

    def display(self, ind=1):
        print('  ' * ind, self.itemName, ' ', self.count)
        for child in list(self.children.values()):
            child.display(ind+1)

In [1664]:
def updateHeaderTable(item, targetNode, headerTable):
    if(headerTable[item][1] == None):
        headerTable[item][1] = targetNode
    else:
        currentNode = headerTable[item][1]
        # Traverse to the last node then link it to the target
        while currentNode.next != None:
            currentNode = currentNode.next
        currentNode.next = targetNode

In [1665]:
def updateTree(item, parentNode, headerTable):
    if item in parentNode.children:
        # If the item already exists, increment the count
        parentNode.children[item].increment(1)
    else:
        # Create a new branch
        newItemNode = Node(item, 1, parentNode)
        parentNode.children[item] = newItemNode
        # Link the new branch to header table
        updateHeaderTable(item, newItemNode, headerTable)

    return parentNode.children[item]

In [1666]:
def constructTree(listOfItemset, frequency, minimumSupport):
    headerTable = defaultdict(int)
    # Counting frequency and create header table
    for idx, itemSet in enumerate(listOfItemset):
        for item in itemSet:
            headerTable[item] += frequency[idx]

    # Deleting items below minSup
    headerTable = dict((item, sup) for item, sup in headerTable.items() if sup >= minimumSupport)

    if(len(headerTable) == 0):
        return None, None

    # HeaderTable column [Item: [frequency, headNode]]
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    # Init Null head node
    initialNode = Node('Null', 1, None)
    # Update FP tree for each cleaned and sorted itemSet
    for _, itemset in enumerate(listOfItemset):
        itemset = [item for item in itemset if item in headerTable]
        itemset.sort(key=lambda item: (-headerTable[item][0], item))

        # Traverse from root to leaf, update tree with given item
        currentNode = initialNode
        for item in itemset:
            currentNode = updateTree(item, currentNode, headerTable)

    return initialNode, headerTable

### Mining FP-Tree

In [1667]:
def findPrefixPath(node, prefixPath):
    if node.parent != None:
        prefixPath.append(node.itemName)
        findPrefixPath(node.parent, prefixPath)

In [1668]:
def createConditionalPatternBase(item, headerTable):
    # First node in linked list
    nodeOfTree = headerTable[item][1]
    conditionalPaths = []
    frequencyOfEachPath = []

    while nodeOfTree != None:
        prefixPath = []
        # From leaf node all the way to root and reverse the prefix path
        findPrefixPath(nodeOfTree, prefixPath)
        prefixPath = prefixPath[::-1]
        
        if len(prefixPath) > 1:
            # Storing the prefix path and it's corresponding count
            conditionalPaths.append(prefixPath[:len(prefixPath)-1])
            frequencyOfEachPath.append(nodeOfTree.count)

        # Go to next node / item in other branch
        nodeOfTree = nodeOfTree.next

    return conditionalPaths, frequencyOfEachPath

In [1669]:
def constructConditionalTree(conditionalPatternBase, frequency, minimumSupport):
    conditionalHeaderTable = defaultdict(int)
    # Counting frequency and create header table
    for idx, itemSet in enumerate(conditionalPatternBase):
        for item in itemSet:
            conditionalHeaderTable[item] += frequency[idx]

    # Deleting items below minSup
    conditionalHeaderTable = dict((item, sup) for item, sup in conditionalHeaderTable.items() if sup >= minimumSupport)

    if(len(conditionalHeaderTable) == 0):
        return None, None

    # HeaderTable column [Item: [frequency, headNode]]
    for item in conditionalHeaderTable:
        conditionalHeaderTable[item] = [conditionalHeaderTable[item], None]

    # Init Null head node
    conditionalInitialNode = Node('Null', 1, None)

    conditionalPatternBaseExtracted = []
    for itemset, freq in zip(conditionalPatternBase, frequency):
        conditionalPatternBaseExtracted.extend([itemset.copy() for _ in range(freq)])

    # Update FP tree for each cleaned and sorted itemSet
    for _, itemset in enumerate(conditionalPatternBaseExtracted):
        itemset = [item for item in itemset if item in conditionalHeaderTable]

        # Traverse from root to leaf, update tree with given item
        currentNode = conditionalInitialNode
        for item in itemset:
            currentNode = updateTree(item, currentNode, conditionalHeaderTable)

    return conditionalInitialNode, conditionalHeaderTable

In [1670]:
def mineTree(headerTable, minSup, preFix, freqItemList):
    # Sort the items with frequency and create a list
    # sortedItemList = [item[0] for item in sorted(list(headerTable.items()), key=lambda p:p[1][0])]
    sortedItemList = [item[0] for item in sorted(headerTable.items(), key=lambda item: (-item[1][0], item[0]))]
    reversedItemList = sortedItemList[::-1]

    # Start with the lowest frequency
    for item in reversedItemList:  
        # Pattern growth is achieved by the concatenation of suffix pattern with frequent patterns generated from conditional FP-tree
        newFreqSet = preFix.copy()
        newFreqSet.add(item)
        freqItemList.append(newFreqSet)

        # Find all prefix path, constrcut conditional pattern base
        conditionalPattBase, frequency = createConditionalPatternBase(item, headerTable)
        # Construct conditonal FP Tree with conditional pattern base
        print("conditionalPattBase item", item, conditionalPattBase)
        conditionalTree, newHeaderTable = constructConditionalTree(conditionalPattBase, frequency, minSup)
        if newHeaderTable != None:
            print(conditionalTree.display())
            # Mining recursively on the tree
            mineTree(newHeaderTable, minSup, newFreqSet, freqItemList)

In [1671]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

In [1672]:
def getSupport(testSet, itemSetList):
    count = 0
    for itemSet in itemSetList:
        if(set(testSet).issubset(itemSet)):
            count += 1
    return count

In [1673]:
def associationRule(freqItemSet, itemSetList, minConf):
    rules = []
    for itemSet in freqItemSet:
        subsets = powerset(itemSet)
        itemSetSup = getSupport(itemSet, itemSetList)
        for s in subsets:
            confidence = float(itemSetSup / getSupport(s, itemSetList))
            if(confidence > minConf):
                rules.append([set(s), set(itemSet.difference(s)), confidence])
    return rules

In [1674]:
def getFrequencyFromList(itemSetList):
    frequency = [1 for i in range(len(itemSetList))]
    return frequency

### Trigger the Algorithm

In [1675]:
def getDatasetFromFile(fname):
    listOfItemset = []
    frequencyOfTransaction = []
    
    with open(fname, 'r') as file:
        csv_reader = reader(file)
        for line in csv_reader:
            line = list(filter(None, line))
            listOfItemset.append(line)
            frequencyOfTransaction.append(1)

    return listOfItemset, frequencyOfTransaction

In [1676]:
def fpgrowthFromFile(fname, minSupRatio, minConf):
    listOfItemset, frequency = getDatasetFromFile(fname)
    minimumSupport = len(listOfItemset) * minSupRatio

    fpTree, headerTable = constructTree(listOfItemset, frequency, minimumSupport)
    # print(fpTree.display())
    if(fpTree.children == None):
        print('No frequent item set')
    else:
        freqItems = []
        mineTree(headerTable, minimumSupport, set(), freqItems)
        rules = associationRule(freqItems, listOfItemset, minConf)
        return freqItems, rules

In [1677]:
freqItems, rules = fpgrowthFromFile("data/example.csv", 0.6, 0.1)
# print("\n")
# print("Frequent pattern itemset: ")
freqItems

conditionalPattBase item p [['c', 'f', 'a', 'm'], ['c', 'b']]
   Null   1
     c   3
None
conditionalPattBase item c []
conditionalPattBase item m [['c', 'f', 'a'], ['c', 'f', 'a', 'b']]
   Null   1
     c   3
       f   3
         a   3
None
conditionalPattBase item f [['c']]
   Null   1
     c   3
None
conditionalPattBase item c []
conditionalPattBase item c []
conditionalPattBase item a [['c', 'f']]
   Null   1
     c   3
       f   3
None
conditionalPattBase item f [['c']]
   Null   1
     c   3
None
conditionalPattBase item c []
conditionalPattBase item c []
conditionalPattBase item b [['c', 'f', 'a'], ['f'], ['c']]
conditionalPattBase item a [['c', 'f']]
   Null   1
     c   3
       f   3
None
conditionalPattBase item f [['c']]
   Null   1
     c   3
None
conditionalPattBase item c []
conditionalPattBase item c []
conditionalPattBase item f [['c']]
   Null   1
     c   3
None
conditionalPattBase item c []
conditionalPattBase item c []


[{'p'},
 {'c', 'p'},
 {'m'},
 {'f', 'm'},
 {'c', 'f', 'm'},
 {'c', 'm'},
 {'a', 'm'},
 {'a', 'f', 'm'},
 {'a', 'c', 'f', 'm'},
 {'a', 'c', 'm'},
 {'b'},
 {'a'},
 {'a', 'f'},
 {'a', 'c', 'f'},
 {'a', 'c'},
 {'f'},
 {'c', 'f'},
 {'c'}]